# Libraries

In [1]:
import  pandas            as pd
import  numpy             as np
import  plotly.express    as px
import  matplotlib.pyplot as plt
import  plotly.graph_objs as go
import  plotly.io         as pio

In [2]:
RANDOM_SEED = np.random.seed(31)


In [3]:
cd ..\

c:\Users\victo\Downloads\00. Estudo Data Science\predict_payment_default


c:\Users\victo\Downloads\00. Estudo Data Science\predict_payment_default\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# db info

In [4]:
df_info = pd.read_csv('data\database_info.csv',
                      encoding='utf-8',
                      sep=','
                      )
df_info['ID_CLIENTE'] = df_info['ID_CLIENTE'].astype(object)
df_info['SAFRA_REF'] = pd.to_datetime(df_info['SAFRA_REF']).dt.to_period('M')
df_info.sort_values(['ID_CLIENTE', 'SAFRA_REF'], inplace=True)
df_info.reset_index(inplace=True, drop=True)
df_info.head()

,ID_CLIENTE,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS
0,8784237149961904,2018-09,300502.0,107.0
1,8784237149961904,2018-10,499238.0,98.0
2,8784237149961904,2018-11,256117.0,104.0
3,8784237149961904,2018-12,433897.0,105.0
4,8784237149961904,2019-01,370549.0,112.0


In [5]:
df_info.shape

(24401, 4)

In [6]:
df_info.dtypes

ID_CLIENTE               object
SAFRA_REF             period[M]
RENDA_MES_ANTERIOR      float64
NO_FUNCIONARIOS         float64
dtype: object

In [7]:
df_info.isna().sum()

ID_CLIENTE            0
SAFRA_REF             0
RENDA_MES_ANTERIOR    0
NO_FUNCIONARIOS       0
dtype: int64

In [8]:
df_info.describe()

,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS
count,2.440100e+04,24401.000000
mean,2.885432e+05,117.820540
std,2.114604e+05,21.439451
min,1.050000e+02,0.000000
25%,1.336420e+05,106.000000
50%,2.410160e+05,118.000000
75%,3.922190e+05,131.000000
max,1.682759e+06,198.000000


# db registration

In [9]:
df_regist = pd.read_csv('data\database_registration.csv',
                        encoding='utf-8',
                        sep=','
                        )

map_flag_pf = {'X': 'PJ', np.nan: 'PF'}
df_regist['FLAG_PF'] = df_regist['FLAG_PF'].map(map_flag_pf)

df_regist['CEP_2_DIG'] = df_regist['CEP_2_DIG'].replace({'na': np.nan})
df_regist['CEP_2_DIG']  = df_regist['CEP_2_DIG'] .astype(float)

# Adjust wrong data on DDD to np.nan
df_regist['DDD'] = df_regist['DDD'].apply(lambda x: np.nan if pd.isna(x) else (np.nan if '(' in x or '00' in x or x.startswith('0') else x))
df_regist['DDD'] = df_regist['DDD'].astype(float)

df_regist['DATA_CADASTRO'] = pd.to_datetime(df_regist['DATA_CADASTRO'])
df_regist.head()

,ID_CLIENTE,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG
0,1661240395903230676,2013-08-22,99.0,PF,Serviços,YAHOO,PEQUENO,65.0
1,8274986328479596038,2017-01-25,31.0,PF,Comércio,YAHOO,MEDIO,77.0
2,345447888460137901,2000-08-15,75.0,PF,Serviços,HOTMAIL,PEQUENO,48.0
3,1003144834589372198,2017-08-06,49.0,PF,Serviços,OUTLOOK,PEQUENO,89.0
4,324916756972236008,2011-02-14,88.0,PF,Serviços,GMAIL,GRANDE,62.0


In [10]:
df_regist.shape

(1315, 8)

In [11]:
df_regist.dtypes

ID_CLIENTE                      int64
DATA_CADASTRO          datetime64[ns]
DDD                           float64
FLAG_PF                        object
SEGMENTO_INDUSTRIAL            object
DOMINIO_EMAIL                  object
PORTE                          object
CEP_2_DIG                     float64
dtype: object

In [12]:
df_regist.isna().sum()

ID_CLIENTE               0
DATA_CADASTRO            0
DDD                    338
FLAG_PF                  0
SEGMENTO_INDUSTRIAL     83
DOMINIO_EMAIL           30
PORTE                   41
CEP_2_DIG                1
dtype: int64

In [13]:
df_regist['DDD'].value_counts()

DDD
11.0    125
19.0     50
21.0     39
41.0     37
31.0     33
       ... 
87.0      3
79.0      3
52.0      1
56.0      1
95.0      1
Name: count, Length: 65, dtype: int64

In [14]:
df_regist['DOMINIO_EMAIL'].value_counts()

DOMINIO_EMAIL
YAHOO      392
GMAIL      390
HOTMAIL    323
AOL         74
OUTLOOK     72
BOL         34
Name: count, dtype: int64

In [15]:
df_regist['FLAG_PF'].value_counts()

FLAG_PF
PF    1249
PJ      66
Name: count, dtype: int64

In [16]:
df_regist['PORTE'].value_counts()

PORTE
MEDIO      522
GRANDE     480
PEQUENO    272
Name: count, dtype: int64

In [17]:
df_regist['CEP_2_DIG'].value_counts()

CEP_2_DIG
13.0    86
78.0    60
89.0    44
68.0    42
29.0    40
        ..
24.0     4
52.0     3
67.0     3
82.0     2
51.0     2
Name: count, Length: 89, dtype: int64

## DDD processing

In [18]:
ddd_states_brazil = {
    'AC': [68],
    'AL': [82],
    'AM': [92, 97],
    'AP': [96],
    'BA': [71, 73, 74, 75, 77],
    'CE': [85, 88],
    'DF': [61],
    'ES': [27, 28],
    'GO': [62, 64],
    'MA': [98, 99],
    'MT': [65, 66],
    'MS': [67],
    'MG': [31, 32, 33, 34, 35, 37, 38],
    'PA': [91, 93, 94],
    'PB': [83],
    'PR': [41, 42, 43, 44, 45, 46],
    'PE': [81, 87],
    'PI': [86, 89],
    'RJ': [21, 22, 24],
    'RN': [84],
    'RS': [51, 53, 54, 55],
    'RO': [69],
    'RR': [95],
    'SC': [47, 48, 49],
    'SP': [11, 12, 13, 14, 15, 16, 17, 18, 19],
    'SE': [79],
    'TO': [63]
}

state_ddd = []
for k1, v1 in ddd_states_brazil.items():
    for v2 in v1:
        state_ddd.append([k1, v2])

df_ddd_brazil = pd.DataFrame(state_ddd, columns=['UF', 'DDD'])
df_ddd_brazil['DDD'] = df_ddd_brazil['DDD'].astype(float)
df_ddd_brazil.head()

,UF,DDD
0,AC,68.0
1,AL,82.0
2,AM,92.0
3,AM,97.0
4,AP,96.0


In [19]:
df_ddd_brazil.dtypes

UF      object
DDD    float64
dtype: object

In [20]:
df_regist['DDD'].dtypes

dtype('float64')

In [21]:
df_regist_ddd = df_regist.set_index('DDD').join(df_ddd_brazil.set_index('DDD'))
df_regist_ddd.reset_index(drop=False, inplace=True)
df_regist_ddd

,DDD,ID_CLIENTE,DATA_CADASTRO,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG,UF
0,99.0,1661240395903230676,2013-08-22,PF,Serviços,YAHOO,PEQUENO,65.0,MA
1,31.0,8274986328479596038,2017-01-25,PF,Comércio,YAHOO,MEDIO,77.0,MG
2,75.0,345447888460137901,2000-08-15,PF,Serviços,HOTMAIL,PEQUENO,48.0,BA
3,49.0,1003144834589372198,2017-08-06,PF,Serviços,OUTLOOK,PEQUENO,89.0,SC
4,88.0,324916756972236008,2011-02-14,PF,Serviços,GMAIL,GRANDE,62.0,CE
...,...,...,...,...,...,...,...,...,...
1310,92.0,3431426889924624821,2020-08-13,PF,Serviços,HOTMAIL,MEDIO,69.0,AM
1311,NaN,5288503299611498087,2020-11-03,PF,Comércio,YAHOO,PEQUENO,13.0,NaN
1312,NaN,957773253650890560,2021-07-05,PF,Comércio,GMAIL,MEDIO,20.0,NaN
1313,NaN,6094038865287329652,2021-07-05,PF,Serviços,GMAIL,GRANDE,48.0,NaN


## CEP processing

In [22]:
ceps_states_brazil = {
    "10": "SP", "11": "SP", "12": "SP", "13": "SP", "14": "SP", "15": "SP", "16": "SP", "17": "SP", "18": "SP", "19": "SP",
    "20": "RJ", "21": "RJ", "22": "RJ", "23": "RJ", "24": "RJ",
    "25": "RJ", "26": "RJ", "27": "RJ", "28": "RJ", "29": "RJ",
    "30": "MG", "31": "MG", "32": "MG", "33": "MG", "34": "MG", "35": "MG", "36": "MG", "37": "MG", "38": "MG", "39": "MG",
    "40": "BA", "41": "BA", "42": "BA", "43": "BA", "44": "BA", "45": "BA", "46": "BA", "47": "BA", "48": "BA", "49": "BA",
    "50": "MS", "51": "MS", "52": "GO", "53": "DF", "54": "RS", "55": "RS", "56": "RS", "57": "AL", "58": "PB", "59": "RN",
    "60": "CE", "61": "CE", "62": "CE", "63": "CE", "64": "CE", "65": "CE", "66": "CE", "67": "CE", "68": "PA", "69": "RO",
    "70": "DF", "71": "DF", "72": "DF", "73": "DF", "74": "DF", "75": "DF", "76": "DF", "77": "DF", "78": "MT", "79": "SE",
    "80": "PR", "81": "PR", "82": "PR", "83": "PR", "84": "PR", "85": "PR", "86": "PR", "87": "PR", "88": "SC", "89": "SC",
    "90": "RS", "91": "RS", "92": "RS", "93": "RS", "94": "RS", "95": "RS", "96": "AP", "97": "AM", "98": "MA", "99": "MA"
}

In [23]:
df_ceps_brasil = pd.DataFrame(list(ceps_states_brazil.items()), columns=['CEP_2_DIG', 'UF'])
df_ceps_brasil['CEP_2_DIG'] = df_ceps_brasil['CEP_2_DIG'].astype(float)
df_ceps_brasil.head()

,CEP_2_DIG,UF
0,10.0,SP
1,11.0,SP
2,12.0,SP
3,13.0,SP
4,14.0,SP


In [24]:
df_regist_states = df_regist_ddd.set_index('CEP_2_DIG').join(df_ceps_brasil.set_index('CEP_2_DIG'),
                                                             rsuffix='_DDD',
                                                             lsuffix='_CEP'
                                                             )
df_regist_states.reset_index(drop=False, inplace=True)
df_regist_states.head()

,CEP_2_DIG,DDD,ID_CLIENTE,DATA_CADASTRO,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,UF_CEP,UF_DDD
0,65.0,99.0,1661240395903230676,2013-08-22,PF,Serviços,YAHOO,PEQUENO,MA,CE
1,77.0,31.0,8274986328479596038,2017-01-25,PF,Comércio,YAHOO,MEDIO,MG,DF
2,48.0,75.0,345447888460137901,2000-08-15,PF,Serviços,HOTMAIL,PEQUENO,BA,BA
3,89.0,49.0,1003144834589372198,2017-08-06,PF,Serviços,OUTLOOK,PEQUENO,SC,SC
4,62.0,88.0,324916756972236008,2011-02-14,PF,Serviços,GMAIL,GRANDE,CE,CE


In [25]:
count_states_cep = df_regist_states['UF_CEP'].value_counts()
count_states_ddd = df_regist_states['UF_DDD'].value_counts()

df_count_states = pd.DataFrame([count_states_cep, count_states_ddd],
                                index=['count_states_cep', 'count_states_ddd']).T
df_count_states.head()

,count_states_cep,count_states_ddd
SP,267.0,218.0
MG,109.0,155.0
PR,93.0,122.0
RS,71.0,83.0
RJ,58.0,142.0


In [26]:
df_count_states

,count_states_cep,count_states_ddd
SP,267.0,218.0
MG,109.0,155.0
PR,93.0,122.0
RS,71.0,83.0
RJ,58.0,142.0
SC,57.0,72.0
BA,50.0,96.0
GO,39.0,3.0
MT,36.0,60.0
PA,30.0,42.0


In [27]:
# Customers per state (CEP)
fig = px.bar(y=df_count_states.sort_values('count_states_cep', ascending=False)['count_states_cep'],
             x=df_count_states.sort_values('count_states_cep', ascending=False)['count_states_cep'].index,
             text_auto=True
             )
fig.update_traces(textfont=dict(size=16))
fig.update_layout(height=400,
                  width=800,
                  showlegend=False,
                  title='Customers per state (CEP)',
                  margin=dict(l=10, r=10, b=10, t=30, pad=0),
                  xaxis=dict(title='State', tickfont=dict(size=14)),
                  yaxis=dict(title='Registers', titlefont=dict(size=16))
                  )
fig.show()

In [28]:
# Customers per state (DDD)
fig = px.bar(y=df_count_states.sort_values('count_states_ddd', ascending=False)['count_states_ddd'],
             x=df_count_states.sort_values('count_states_ddd', ascending=False)['count_states_ddd'].index,
             text_auto=True
             )
fig.update_traces(textfont=dict(size=16))
fig.update_layout(height=400,
                  width=800,
                  showlegend=False,
                  title='Customers per state (DDD)',
                  margin=dict(l=10, r=10, b=10, t=30, pad=0),
                  xaxis=dict(title='State', tickfont=dict(size=14)),
                  yaxis=dict(title='Registers', titlefont=dict(size=16))
                  )
fig.show()

In [29]:
df_count_states

,count_states_cep,count_states_ddd
SP,267.0,218.0
MG,109.0,155.0
PR,93.0,122.0
RS,71.0,83.0
RJ,58.0,142.0
SC,57.0,72.0
BA,50.0,96.0
GO,39.0,3.0
MT,36.0,60.0
PA,30.0,42.0


In [30]:
# customers per state (DDD)
fig = px.choropleth(df_count_states, 
                    geojson='https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson', 
                    locations=df_count_states.index, 
                    featureidkey='properties.sigla', 
                    color='count_states_ddd',
                    color_continuous_scale='viridis_r',
                    scope='south america'
                    )

fig.update_geos(fitbounds='locations', visible=False)
fig.update_layout(height=600,
                  width=600,
                  showlegend=True,
                  margin=dict(l=10, r=10, b=10, t=30, pad=10),
                  title='customers per state (DDD)'
                  )
fig.show()

In [31]:
# customers per state (CEP)
fig = px.choropleth(df_count_states, 
                    geojson='https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson', 
                    locations=df_count_states.index, 
                    featureidkey='properties.sigla', 
                    color='count_states_cep',
                    color_continuous_scale='viridis_r',
                    scope='south america'
                    )

fig.update_geos(fitbounds='locations', visible=False)
fig.update_layout(height=600,
                  width=600,
                  showlegend=True,
                  margin=dict(l=10, r=10, b=10, t=30, pad=10),
                  title='customers per state (CEP)'
                  )
fig.show()

# db dev

In [32]:
df_dev = pd.read_csv('data\database_dev.csv',
                     encoding='utf-8',
                     sep=','
                     )

df_dev['ID_CLIENTE'] = df_dev['ID_CLIENTE'].astype(object)
df_dev['SAFRA_REF'] = pd.to_datetime(df_dev['SAFRA_REF']).dt.to_period('M')
df_dev['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(df_dev['DATA_EMISSAO_DOCUMENTO'])
df_dev['DATA_PAGAMENTO'] = pd.to_datetime(df_dev['DATA_PAGAMENTO'])
df_dev['DATA_VENCIMENTO'] = pd.to_datetime(df_dev['DATA_VENCIMENTO'])
df_dev['DIAS_ATRASO'] = (df_dev['DATA_PAGAMENTO'] \
                                     - df_dev['DATA_VENCIMENTO']).dt.days
df_dev['INADIMP'] = np.where(df_dev['DIAS_ATRASO'] >= 5, 1, 0)

df_dev.sort_values(['ID_CLIENTE', 'SAFRA_REF'], inplace=True)
df_dev.reset_index(inplace=True, drop=True)
df_dev.head()

,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATRASO,INADIMP
0,8784237149961904,2018-08,2018-08-17,2018-09-04,2018-09-04,100616.1,5.99,0,0
1,8784237149961904,2018-08,2018-08-22,2018-09-11,2018-09-11,89552.8,5.99,0,0
2,8784237149961904,2018-08,2018-08-23,2018-09-10,2018-09-10,94062.8,5.99,0,0
3,8784237149961904,2018-08,2018-08-23,2018-09-08,2018-09-10,102686.1,5.99,-2,0
4,8784237149961904,2018-08,2018-08-24,2018-09-11,2018-09-11,51393.0,5.99,0,0


In [33]:
df_dev.shape

(77414, 9)

In [34]:
df_dev.dtypes

ID_CLIENTE                        object
SAFRA_REF                      period[M]
DATA_EMISSAO_DOCUMENTO    datetime64[ns]
DATA_PAGAMENTO            datetime64[ns]
DATA_VENCIMENTO           datetime64[ns]
VALOR_A_PAGAR                    float64
TAXA                             float64
DIAS_ATRASO                        int64
INADIMP                            int64
dtype: object

In [35]:
df_dev.isna().sum()

ID_CLIENTE                0
SAFRA_REF                 0
DATA_EMISSAO_DOCUMENTO    0
DATA_PAGAMENTO            0
DATA_VENCIMENTO           0
VALOR_A_PAGAR             0
TAXA                      0
DIAS_ATRASO               0
INADIMP                   0
dtype: int64

In [36]:
df_dev['INADIMP'].value_counts()

INADIMP
0    71978
1     5436
Name: count, dtype: int64

In [37]:
df_dev[['VALOR_A_PAGAR', 'TAXA', 'DIAS_ATRASO']].describe()

,VALOR_A_PAGAR,TAXA,DIAS_ATRASO
count,7.741400e+04,77414.000000,77414.000000
mean,4.656546e+04,6.789623,-0.171429
std,4.633892e+04,1.798225,25.229477
min,1.000000e-01,4.990000,-2661.000000
25%,1.875275e+04,5.990000,0.000000
50%,3.475135e+04,5.990000,0.000000
75%,6.088421e+04,6.990000,0.000000
max,4.400000e+06,11.990000,869.000000


In [38]:
# Total payments per customer
df_count_ids = df_dev.value_counts('ID_CLIENTE').to_frame('total').sort_index()

# Total default or non default per customer
df_count_payments = df_dev.value_counts(['ID_CLIENTE', 'INADIMP']).to_frame().sort_index()

df_count_payments.reset_index(level=1, drop=False, inplace=True)
df_pct_payments = df_count_ids.join(df_count_payments)

# Default percentage per customer
df_pct_payments['pct'] = df_pct_payments['count'] / df_pct_payments['total'] * 100

df_pct_payments.reset_index(drop=False, inplace=True)
df_pct_payments['ID_CLIENTE'] = df_pct_payments['ID_CLIENTE'].astype('object')
df_pct_payments['INADIMP'] = df_pct_payments['INADIMP'].astype('object')
df_pct_payments.head()

,ID_CLIENTE,total,INADIMP,count,pct
0,8784237149961904,241,0,240,99.585062
1,8784237149961904,241,1,1,0.414938
2,15070048310374146,5,0,4,80.000000
3,15070048310374146,5,1,1,20.000000
4,18719614953021044,7,0,6,85.714286


In [39]:
df_defaults = df_pct_payments.sort_values('pct', ascending=False)
df_defaults_over_50 = df_defaults[(df_defaults['INADIMP'] == 1)
                                  & (df_defaults['pct'] > 50)]
df_defaults_over_50.head()

,ID_CLIENTE,total,INADIMP,count,pct
8,43266641219008038,9,1,9,100.0
15,86436955037443683,1,1,1,100.0
781,4068666180981979477,1,1,1,100.0
695,3540559576342636557,1,1,1,100.0
1692,8594897944068249710,7,1,7,100.0


In [40]:
# Customers with default over 50%
fig = px.bar(df_defaults_over_50,
             y='pct',
             x='ID_CLIENTE',
             text_auto=True,
             color='INADIMP',
             barmode='group'
             )
fig.update_traces(textfont=dict(size=16))
fig.update_layout(height=400,
                  width=800,
                  showlegend=False,
                  title='Customers with default over 50%',
                  margin=dict(l=10, r=10, b=10, t=30, pad=0),
                  xaxis=dict(title='ID Customer', tickfont=dict(size=14)),
                  yaxis=dict(title='% of default', titlefont=dict(size=16))
                  )
fig.update_xaxes(type='category')
fig.show()